In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print("Tensorflow version: " + tf.__version__)

Tensorflow version: 2.0.0


In [11]:
def draw_sequence (n):
    
    return np.random.uniform(0,9,n) 

In [8]:
#Part (a):

data = np.array([draw_sequence(30) for i in range(10000)])
labels = np.apply_along_axis(lambda x: np.sum(x) >= 100, 1, data)

x_train, y_train = data[0:8000], labels[0:8000]*1
x_valid, y_valid = data[8000:10001], labels[8000:10001]*1

#Rescale the input dimension to fit the LSTM:

x1,x2 = x_train.shape[0], x_valid.shape[0]
y1,y2 = x_train.shape[1], x_valid.shape[1]
x_train = np.reshape(x_train, (x1, y1, 1))
x_valid = np.reshape(x_valid, (x2, y2, 1))

In [23]:
#Part (b): Build a recurrent network

seq_len, time_steps = x_train.shape[1], 30
batch_size = 60

lstm_model = tf.keras.models.Sequential([
    
    tf.keras.layers.LSTM(200, input_shape = (time_steps, seq_len)),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
   
])

lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 200)               184800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 185,001
Trainable params: 185,001
Non-trainable params: 0
_________________________________________________________________


In [10]:
#Compile the model:

adam = tf.keras.optimizers.Adam(learning_rate=.001, 
                             beta_1= .9, 
                             beta_2= .999, 
                             epsilon = 1e-08,
                             amsgrad=False)

lstm_model.compile(optimizer= adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
print(x_train.shape)

#Fit the model:

lstm_model.fit(x_train, y_train, batch_size = 50, epochs = 10)

#Evaluate the model:

lstm_model.evaluate(x_train, y_train, verbose = 2) #on training set
lstm_model.evaluate(x_valid, y_valid, verbose=2) #on hold-out set

(8000, 30, 1)


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [121]:
#Exploring the class distribution:

print(np.mean(y_train), np.mean(y_valid))

0.992375 0.989


$\textit{Part (a)}$: The final accuracy of the model is pretty high $\approx$ 99.85 %. But we should be careful to trust the prediction of the model. When we look at the actual distribution of labels within our training and validation sets, we find that it's actually pretty close to the performances on both the training and validations sets as measured by accuracy. So the model could get its predictive power just by correctly predicting on the positive examples. In other words, the model could totally malperform on the negative labels. We also tried out different alternatives. For example, using the cosine similarity metric we got results that exactly matched the percentage of positive labels in the training and validation sets respectively. 